# Notebook to run basic `scanpy` QC and doublet detection with `scrublet` for SRA Project - PRJNA762100
- **Developed by**: Srivalli Kolla

- **Created date** : 14 October, 2024

- **Modification date** : 23 October, 2024

- **Würzburg Institute for Systems Immunology & Julius-Maximilian-Universität Würzburg**

Env : Scanpy(Python 3.12.4)

# Import sample_names

In [1]:
import anndata
import logging
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sb
import scrublet as scr
import os
import time
import matplotlib.pyplot as plt
from statsmodels.robust.scale import mad as median_abs_deviation
from matplotlib import colors
from matplotlib import rcParams

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 160, color_map = 'RdPu', dpi_save = 180, vector_friendly = True, format = 'svg')
timestamp = time.strftime("%d_%m_%Y")

# Import sample_names

In [3]:
path = '../ncbi_sra/data'
files = ['SRR15835820.h5ad',
'SRR15835821.h5ad',
'SRR15835824.h5ad',
'SRR15835825.h5ad',
'SRR15835830.h5ad',
'SRR15835831.h5ad',
'SRR15835875.h5ad',
'SRR15835876.h5ad',
'SRR15835877.h5ad',
'SRR15835878.h5ad',
'SRR15835879.h5ad',
'SRR15835880.h5ad',
'SRR15835881.h5ad',
'SRR15835882.h5ad',
'SRR15835883.h5ad',
'SRR15835884.h5ad',
'SRR15835885.h5ad',
'SRR15835886.h5ad',
'SRR15835887.h5ad',
'SRR15835888.h5ad',
'SRR15835889.h5ad',
'SRR15835890.h5ad',
'SRR15835891.h5ad',
'SRR15835892.h5ad',
'SRR15835893.h5ad',
'SRR15835894.h5ad',
'SRR15835895.h5ad',
'SRR15835896.h5ad',
'SRR15835897.h5ad',
'SRR15835898.h5ad',
'SRR15835899.h5ad',
'SRR15835900.h5ad',
'SRR15835901.h5ad',
'SRR15835902.h5ad',
'SRR15835903.h5ad',
'SRR15835904.h5ad',
'SRR15835905.h5ad',
'SRR15835906.h5ad',
'SRR15835907.h5ad',
'SRR15835908.h5ad',
'SRR15835909.h5ad',
'SRR15835910.h5ad',
'SRR15835911.h5ad',
'SRR15835912.h5ad',
'SRR15835913.h5ad',
'SRR15835914.h5ad',
'SRR15835915.h5ad',
'SRR15835917.h5ad',
'SRR15835919.h5ad',
'SRR15835813.h5ad',
'SRR15835816.h5ad',
'SRR15835818.h5ad',
'SRR15835819.h5ad',
'SRR15835822.h5ad',
'SRR15835823.h5ad',
'SRR15835826.h5ad',
'SRR15835827.h5ad',
'SRR15835828.h5ad',
'SRR15835829.h5ad',
'SRR15835916.h5ad',
'SRR15835918.h5ad',
'SRR15835920.h5ad',
'SRR15835921.h5ad',
'SRR15835922.h5ad',
'SRR15835923.h5ad',
'SRR15835812.h5ad',
'SRR15835814.h5ad',
'SRR15835815.h5ad',
'SRR15835817.h5ad',
'SRR15835832.h5ad',
'SRR15835833.h5ad',
'SRR15835834.h5ad',
'SRR15835835.h5ad',
'SRR15835836.h5ad',
'SRR15835837.h5ad',
'SRR15835838.h5ad',
'SRR15835839.h5ad',
'SRR15835840.h5ad',
'SRR15835841.h5ad',
'SRR15835842.h5ad',
'SRR15835843.h5ad',
'SRR15835844.h5ad',
'SRR15835845.h5ad',
'SRR15835846.h5ad',
'SRR15835847.h5ad',
'SRR15835848.h5ad',
'SRR15835849.h5ad',
'SRR15835850.h5ad',
'SRR15835851.h5ad',
'SRR15835852.h5ad',
'SRR15835853.h5ad',
'SRR15835854.h5ad',
'SRR15835855.h5ad',
'SRR15835856.h5ad',
'SRR15835857.h5ad',
'SRR15835858.h5ad',
'SRR15835859.h5ad',
'SRR15835860.h5ad',
'SRR15835861.h5ad',
'SRR15835862.h5ad',
'SRR15835863.h5ad',
'SRR15835864.h5ad',
'SRR15835865.h5ad',
'SRR15835866.h5ad',
'SRR15835867.h5ad',
'SRR15835868.h5ad',
'SRR15835869.h5ad',
'SRR15835870.h5ad',
'SRR15835871.h5ad',
'SRR15835872.h5ad',
'SRR15835873.h5ad',
'SRR15835874.h5ad']

In [ ]:
adata_combined = None
vars_combined = []

for file in files:
    file_path = os.path.join(path, file)
    
    try:

        adata = sc.read_h5ad(file_path)

        sample_name = os.path.basename(file_path).split('.')[0]
        adata.obs['sample_name'] = sample_name


        sc.pp.filter_cells(adata, min_counts=10)
        sc.pp.filter_genes(adata, min_counts=10)


        adata.var_names = adata.var_names.str.split('.').str[0]
        adata.var_names = [f"{name}_{sample_name}" for name in adata.var_names]

        vars_combined.extend(adata.var_names)


        if adata_combined is None:
            adata_combined = adata
        else:
            adata_combined = sc.concat([adata_combined, adata], join='outer', index_unique='-')

        print(f"Successfully read and concatenated: {file}")

    except Exception as e:
        print(f"Error reading {file}: {e}")

unique_var_names = pd.Series(vars_combined).unique()
adata_combined.var_names = unique_var_names[:adata_combined.n_vars] 
adata_combined

In [ ]:
adata_combined

In [ ]:
adata_combined.obs

In [ ]:
adata_combined.var

In [ ]:
adata_combined.obs['sample_name'].value_counts()

## Doublet score prediction

In [ ]:
scrub = scr.Scrublet(adata_combined.X)

doublet_scores, predicted_doublets = scrub.scrub_doublets()
            
adata_combined.obs['doublet_scores'] = doublet_scores
adata_combined.obs['predicted_doublets'] = predicted_doublets

In [ ]:
adata_combined.obs

### Checking the count and percentage of Doublets - sample_name level

In [ ]:
doub_tab = pd.crosstab(adata_combined.obs['sample_name'],adata_combined.obs['predicted_doublets'])
doub_tab.sum()

In [ ]:
true_doublets = adata_combined.obs['predicted_doublets'] == True
true_doublets_count = true_doublets.sum()

true_doublets_percentage = (true_doublets_count / len(adata_combined.obs)) * 100

true_doublets_count ,true_doublets_percentage

### Saving raw data

In [ ]:
sample_name_object = adata_combined.copy()
sample_name_object

## Compute QC stats

In [ ]:
sample_name_object.shape

### Labelling Mt and Ribo genes

In [ ]:
sample_name_object.var

In [ ]:
sample_name_object.var.index = sample_name_object.var.index.str.split('_').str[0]
sample_name_object.var

In [ ]:
sample_name_object.var['ensembl'] = sample_name_object.var.index
sample_name_object.var 

### Ensembl annotations

In [18]:
annot = sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "external_gene_name", "start_position", "end_position", "chromosome_name"],
    ).set_index("ensembl_gene_id")

In [ ]:
annot.head()

In [ ]:
sample_name_object.var

In [ ]:
sample_name_object.var['gene_name'] = sample_name_object.var.index.map(annot['external_gene_name'])
sample_name_object.var.index =sample_name_object.var['gene_name'] 
sample_name_object.var

In [ ]:
sample_name_object.var['mt'] = sample_name_object.var_names.str.startswith('MT-') 
sample_name_object.var['ribo'] = sample_name_object.var_names.str.startswith(("RPS","RPL"))
sample_name_object.var

In [ ]:
ribo_counts = sample_name_object.var['ribo'].value_counts()

mt_counts = sample_name_object.var['mt'].value_counts()

print("Counts of Ribosomal (ribo) Genes:")
print("False:", ribo_counts.get(False, 0))
print("True:", ribo_counts.get(True, 0))
print("\nCounts of Mitochondrial (mt) Genes:")
print("False:", mt_counts.get(False, 0))
print("True:", mt_counts.get(True, 0))

In [ ]:
sample_name_object.var['mt'] = sample_name_object.var['mt'].fillna(False)
sample_name_object.var['ribo'] = sample_name_object.var['ribo'].fillna(False)

### Calculating QC metrics per cell

In [25]:
sc.pp.calculate_qc_metrics(sample_name_object,qc_vars = ['mt','ribo'],inplace = True)

In [ ]:
sample_name_object

## Sex covariate analysis

### Chr Y genes calculation

In [27]:
sample_name_object.var['gene_name'] = sample_name_object.var['ensembl'].map(annot['external_gene_name'])
sample_name_object.var['chromosome'] = sample_name_object.var['ensembl'].map(annot['chromosome_name'])

In [ ]:
sample_name_object.var

In [ ]:
chrY_genes = sample_name_object.var['chromosome'] == "Y"
chrY_genes

In [ ]:
sample_name_object.obs['percent_chrY'] = np.sum(
    sample_name_object[:, chrY_genes].X, axis = 1) / np.sum(sample_name_object.X, axis = 1) * 100

In [ ]:
sample_name_object

### XIST counts

In [ ]:
sample_name_object.var_names

In [ ]:
valid_var_names = sample_name_object.var_names[~sample_name_object.var_names.isna()]

xist_genes = valid_var_names[valid_var_names.str.match('XIST')]

xist_genes

## Calculate cell cycle scores

### Downloading the list of cell cycle genes

In [34]:
!if [ ! -f ../ncbi_sra/data/regev_lab_cell_cycle_genes.txt ]; then curl -o ../ncbi_sra/data/regev_lab_cell_cycle_genes.txt https://raw.githubusercontent.com/theislab/scanpy_usage/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt; fi

### Marking cell cycle genes

#### Steps followed

1. Loading genes and captilizing 
2. Printing the length of cell cycle genes list
3. Split genes into 2 lists (#First 43 genes,#Gene 43 to end)
4. Filtering cell cycle genes only if present in processed_gene_names
5. Print the list of cell cycle genes observed in our data

In [ ]:
cell_cycle_genes = [x.strip() for x in open('../ncbi_sra/data/regev_lab_cell_cycle_genes.txt')]
#cell_cycle_genes = [gene.capitalize() for gene in cell_cycle_genes]
print(len(cell_cycle_genes))

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

cell_cycle_genes = [x for x in cell_cycle_genes if x in sample_name_object.var_names]
print(len(cell_cycle_genes))

In [ ]:
cell_cycle_genes

### Creating basic anndata and normalization for cell cycle score calculation

In [ ]:
adata_combined_log = anndata.AnnData(X = sample_name_object.X,  var = sample_name_object.var, obs = sample_name_object.obs)
sc.pp.normalize_total(adata_combined_log, target_sum = 1e6, exclude_highly_expressed = True)
sc.pp.log1p(adata_combined_log)

### Cell cycle score calculation

In [38]:
adata_combined_log.var_names = adata_combined_log.var_names.astype(str)
adata_combined_log.var_names_make_unique()

In [ ]:
sc.tl.score_genes_cell_cycle(adata_combined_log, s_genes = s_genes, g2m_genes = g2m_genes)

sample_name_object.obs['S_score'] = adata_combined_log.obs['S_score']
sample_name_object.obs['G2M_score'] = adata_combined_log.obs['G2M_score']
sample_name_object.obs['phase'] = adata_combined_log.obs['phase']

sample_name_object

In [ ]:
cell_cycle_counts = sample_name_object.obs['phase'].value_counts()

cell_cycle_counts

In [ ]:
sb.countplot(data=sample_name_object.obs, x='phase')

## Data visualization

In [ ]:
variables = 'n_genes_by_counts', 'total_counts', 'doublet_scores', 'G2M_score', 'S_score'

for var in variables:

    fig, ax = plt.subplots(figsize=(12, 6), ncols=2, gridspec_kw={'width_ratios': [4, 1]})

    sb.violinplot(data=sample_name_object.obs,x = 'sample_name' , y=var, ax=ax[0])
   
    medians = sample_name_object.obs.groupby('sample_name')[var].median()

    for sample_name, median in medians.items():
        ax[0].text(sample_name, median, f'{median:.2f}', ha='center', va='bottom', color='black', fontsize=10)
    
    ax[0].set_title(f'Violin Plot of {var} by sample_name - Before filtering')
    ax[0].set_xlabel('sample_name')
    ax[0].set_ylabel(var)
    ax[0].tick_params(axis='x', rotation=45)

    median_df = pd.DataFrame({'sample_name': medians.index, 'Median': medians.values})

    ax[1].axis('off')
    ax[1].table(cellText=median_df.values, colLabels=median_df.columns, loc='center')
    ax[1].set_title('Median Values')
    
    plt.tight_layout()
    plt.show()


### Visualization of qc metrics

In [ ]:
variables = ['pct_counts_mt', 'pct_counts_ribo']

sb.violinplot(data=sample_name_object.obs[variables])
plt.xticks(rotation=45)
plt.title(f'Mt and Ribo percentages - Before filtering')

In [ ]:
plt.figure(figsize=(10, 6))
sb.scatterplot(data=sample_name_object.obs, x='total_counts', y='n_genes_by_counts' , alpha = 0.4, s=4)
#plt.xticks(range(0, int(max(sample_name_object.obs['total_counts'])) + 1, 3000),rotation=45, fontsize = 10)
#plt.yticks(range(0, int(max(sample_name_object.obs['n_genes_by_counts'])) + 1, 1000),fontsize = 10)
plt.title(f'Counts vs Genes - Before filtering')
plt.show()

### Filtering based on QC metrics

In [ ]:
filtered_object = sample_name_object[sample_name_object.obs['n_genes_by_counts'] > 10]
filtered_object = filtered_object[filtered_object.obs['n_genes_by_counts'] < 1500]

filtered_object = filtered_object[filtered_object.obs['total_counts'] > 10]
filtered_object = filtered_object[filtered_object.obs['total_counts'] < 2000]

filtered_object = filtered_object[filtered_object.obs['pct_counts_mt'] < 60]
filtered_object = filtered_object[filtered_object.obs['pct_counts_ribo'] < 20]

filtered_object = filtered_object[filtered_object.obs['doublet_scores'] < 0.35]

filtered_object

In [ ]:
filtered_object.obs['sample_name'].value_counts()

In [ ]:
variables = ['pct_counts_mt', 'pct_counts_ribo']

sb.violinplot(data=filtered_object.obs[variables])
plt.xticks(rotation=45)
plt.title(f'Mt and Ribo percentages - After filtering')

In [ ]:
sb.set(style = "whitegrid")
covariate_to_visualize = 'total_counts'

plt.figure(figsize = (10, 6))
sb.histplot(data = filtered_object.obs, x = covariate_to_visualize, stat = 'count', common_norm = False)
plt.xlabel(covariate_to_visualize)
plt.ylabel('Abundance')
plt.title(f'Abundance Plot of {covariate_to_visualize} by sample_name - After filtering')
plt.show()

In [ ]:
sb.set(style = "whitegrid")
covariate_to_visualize = 'n_genes_by_counts'

plt.figure(figsize = (10, 6))
sb.histplot(data = filtered_object.obs, x = covariate_to_visualize, stat = 'count', common_norm = False)
plt.xlabel(covariate_to_visualize)
plt.ylabel('Abundance')
plt.title(f'Abundance Plot of {covariate_to_visualize} by sample_name - After filtering')
plt.show()

In [ ]:
variables = 'n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo', 'doublet_scores', 'G2M_score', 'S_score' 

for var in variables:

    fig, ax = plt.subplots(figsize=(12, 6), ncols=2, gridspec_kw={'width_ratios': [4, 1]})

    sb.violinplot(data=filtered_object.obs, x='sample_name', y=var, ax=ax[0])
   
    medians = filtered_object.obs.groupby('sample_name')[var].median()

    for sample_name, median in medians.items():
        ax[0].text(sample_name, median, f'{median:.2f}', ha='center', va='bottom', color='black', fontsize=10)
    
    ax[0].set_title(f'Violin Plot of {var} by sample_name - After filtering')
    ax[0].set_xlabel('sample_name')
    ax[0].set_ylabel(var)
    ax[0].tick_params(axis='x', rotation=45)

    median_df = pd.DataFrame({'sample_name': medians.index, 'Median': medians.values})

    ax[1].axis('off')
    ax[1].table(cellText=median_df.values, colLabels=median_df.columns, loc='center')
    ax[1].set_title('Median Values')
    
    plt.tight_layout()
    plt.show()

## Data Export

In [ ]:
filtered_object.raw = filtered_object.copy()

filtered_object.layers['raw_counts'] = filtered_object.X.copy()

filtered_object.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(filtered_object, inplace = False)["X"]
)

filtered_object

In [52]:
pd.set_option('display.max_rows', None)


In [ ]:
filtered_object.obs['sample_name'].value_counts()

In [ ]:
filtered_object.var.dtypes

In [ ]:
filtered_object.var['mt'].value_counts()

In [56]:
filtered_object.var['mt'] = filtered_object.var['mt'].astype(str)

In [ ]:
print(filtered_object.var.dtypes)

In [ ]:
filtered_object.var

In [ ]:
filtered_object.var = filtered_object.var.rename(columns={'gene_name': 'gene_symbol'})
filtered_object.var = filtered_object.var.reset_index()
filtered_object.var

In [ ]:
filtered_object.raw.var.index.name = 'gene_id'  
filtered_object.var

In [61]:
filtered_object.write_h5ad(f'../ncbi_sra/data/PRJNA762100_sra_filtered_sk_{timestamp}.h5ad')